# Swing Vision Transformation
#### Converting SwingVision data into UCLA Tennis Consulting format
#### Run all cells ONCE; restart Kernel and Run All again if needed

#### TODO
- add firstServeLocation and isLet Columns
- Classification Models for isDropshot, isLob, isApproach - Leo's team

## Notebook Start

In [1]:
import pandas as pd
import numpy as np
import os 
import re

# Option to display max rows/columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Load in data

In [2]:
# Input file name here
your_file_name = 'GovindNanda_CooperWilliams_Harvard.xlsx'
swing_data = pd.read_excel(your_file_name, sheet_name='Shots')
swing_data.shape

(488, 25)

In [3]:
swing_data.head()

,Player,Shot,Type,Stroke,Spin,Speed (MPH),Point,Game,Set,Bounce Depth,Bounce Zone,Bounce Side,Bounce (x),Bounce (y),Hit Depth,Hit Zone,Hit Side,Hit (x),Hit (y),Hit (z),Direction,Result,Favorited,Start Time,Video Time
0,Govind Nanda,1,first_serve,Serve,Kick,71.005798,1,1,1,deep,deuce_out,far,-6.920850,18.294973,deep,deuce,near,0.832136,0.170458,2.628215,out wide,Out,False,23:59:07,9.350000
1,Cooper Willams,2,first_return,Forehand,Slice,27.993538,1,1,1,short,deuce,near,0.240642,9.327007,deep,deuce_alley,far,-4.775480,24.602375,1.123737,down the line,In,False,23:59:08,10.470000
2,Govind Nanda,3,serve_plus_one,Forehand,Flat,36.418350,1,1,1,deep,deuce,far,-1.865687,18.531191,short,deuce,near,1.031473,5.433082,1.670364,cross court,In,False,23:59:11,13.150000
3,Govind Nanda,1,first_serve,Serve,Flat,53.824200,2,1,1,short,deuce,near,2.591806,10.773574,deep,ad,near,-0.580245,0.338901,2.584491,out wide,Net,False,23:59:33,35.349998
4,Govind Nanda,1,first_serve,Serve,Slice,61.779400,2,1,1,short,ad,far,3.506049,16.034611,deep,ad,near,-0.379456,0.337840,2.605358,out wide,In,False,23:59:50,52.980000


### Clean swing vision data

In [4]:
swing_data.query('Stroke == "Feed"').head()

,Player,Shot,Type,Stroke,Spin,Speed (MPH),Point,Game,Set,Bounce Depth,Bounce Zone,Bounce Side,Bounce (x),Bounce (y),Hit Depth,Hit Zone,Hit Side,Hit (x),Hit (y),Hit (z),Direction,Result,Favorited,Start Time,Video Time
26,Govind Nanda,0,none,Feed,Flat,25.114799,7,2,1,short,ad,near,-1.886490,7.651868,short,deuce,far,-2.175474,13.509605,1.195190,---,In,False,00:03:13,255.759995
58,Cooper Willams,0,none,Feed,Flat,20.784662,12,3,1,short,deuce,far,-0.059297,18.219997,out,deuce,near,2.237073,-1.494530,1.326656,---,In,False,00:06:25,447.859985
80,Cooper Willams,0,none,Feed,Flat,15.356256,16,4,1,short,ad,near,-1.723264,9.748523,short,ad,near,-1.679849,5.509582,2.388501,---,Net,False,00:09:35,637.859985
95,Govind Nanda,0,none,Feed,Flat,19.052607,18,4,1,short,deuce,far,-2.948651,16.413523,deep,deuce,near,0.574649,2.620158,0.887377,---,In,False,00:11:42,764.260010
102,Cooper Willams,0,none,Feed,Flat,21.650688,20,5,1,short,ad,near,-1.095460,6.995091,deep,deuce_alley,far,-4.192186,24.912901,1.715653,---,In,False,00:12:26,808.859985


In [5]:
swing_data.query('Shot == 0').head()

,Player,Shot,Type,Stroke,Spin,Speed (MPH),Point,Game,Set,Bounce Depth,Bounce Zone,Bounce Side,Bounce (x),Bounce (y),Hit Depth,Hit Zone,Hit Side,Hit (x),Hit (y),Hit (z),Direction,Result,Favorited,Start Time,Video Time
26,Govind Nanda,0,none,Feed,Flat,25.114799,7,2,1,short,ad,near,-1.886490,7.651868,short,deuce,far,-2.175474,13.509605,1.195190,---,In,False,00:03:13,255.759995
58,Cooper Willams,0,none,Feed,Flat,20.784662,12,3,1,short,deuce,far,-0.059297,18.219997,out,deuce,near,2.237073,-1.494530,1.326656,---,In,False,00:06:25,447.859985
80,Cooper Willams,0,none,Feed,Flat,15.356256,16,4,1,short,ad,near,-1.723264,9.748523,short,ad,near,-1.679849,5.509582,2.388501,---,Net,False,00:09:35,637.859985
95,Govind Nanda,0,none,Feed,Flat,19.052607,18,4,1,short,deuce,far,-2.948651,16.413523,deep,deuce,near,0.574649,2.620158,0.887377,---,In,False,00:11:42,764.260010
102,Cooper Willams,0,none,Feed,Flat,21.650688,20,5,1,short,ad,near,-1.095460,6.995091,deep,deuce_alley,far,-4.192186,24.912901,1.715653,---,In,False,00:12:26,808.859985


In [6]:
swing_data.query('Type == "none"').head()

,Player,Shot,Type,Stroke,Spin,Speed (MPH),Point,Game,Set,Bounce Depth,Bounce Zone,Bounce Side,Bounce (x),Bounce (y),Hit Depth,Hit Zone,Hit Side,Hit (x),Hit (y),Hit (z),Direction,Result,Favorited,Start Time,Video Time
5,Cooper Willams,2,none,Backhand,Flat,77.864120,2,1,1,deep,ad,near,-0.968572,2.478227,deep,ad_alley,far,4.150964,23.764574,1.306965,down the line,In,False,23:59:52,54.080002
6,Govind Nanda,3,none,Forehand,Slice,49.535213,2,1,1,deep,deuce,far,-0.084811,21.055073,deep,ad,near,-1.269274,0.951847,0.991173,inside in,In,False,23:59:53,55.070000
7,Cooper Willams,4,none,Forehand,Flat,72.065048,2,1,1,deep,deuce,near,3.100894,3.324208,deep,ad,far,0.820532,24.497925,1.070541,inside in,In,False,23:59:54,56.520000
26,Govind Nanda,0,none,Feed,Flat,25.114799,7,2,1,short,ad,near,-1.886490,7.651868,short,deuce,far,-2.175474,13.509605,1.195190,---,In,False,00:03:13,255.759995
58,Cooper Willams,0,none,Feed,Flat,20.784662,12,3,1,short,deuce,far,-0.059297,18.219997,out,deuce,near,2.237073,-1.494530,1.326656,---,In,False,00:06:25,447.859985


#### Remove all the rows where it is a feed

In [7]:
rows_to_drop = swing_data.query('Type == "none"').index
swing_data = swing_data.drop(rows_to_drop)
swing_data = swing_data.reset_index(drop=True) # Important to reindex to avoid missing indicies
swing_data.shape

(439, 25)

In [8]:
swing_data.head()

,Player,Shot,Type,Stroke,Spin,Speed (MPH),Point,Game,Set,Bounce Depth,Bounce Zone,Bounce Side,Bounce (x),Bounce (y),Hit Depth,Hit Zone,Hit Side,Hit (x),Hit (y),Hit (z),Direction,Result,Favorited,Start Time,Video Time
0,Govind Nanda,1,first_serve,Serve,Kick,71.005798,1,1,1,deep,deuce_out,far,-6.920850,18.294973,deep,deuce,near,0.832136,0.170458,2.628215,out wide,Out,False,23:59:07,9.350000
1,Cooper Willams,2,first_return,Forehand,Slice,27.993538,1,1,1,short,deuce,near,0.240642,9.327007,deep,deuce_alley,far,-4.775480,24.602375,1.123737,down the line,In,False,23:59:08,10.470000
2,Govind Nanda,3,serve_plus_one,Forehand,Flat,36.418350,1,1,1,deep,deuce,far,-1.865687,18.531191,short,deuce,near,1.031473,5.433082,1.670364,cross court,In,False,23:59:11,13.150000
3,Govind Nanda,1,first_serve,Serve,Flat,53.824200,2,1,1,short,deuce,near,2.591806,10.773574,deep,ad,near,-0.580245,0.338901,2.584491,out wide,Net,False,23:59:33,35.349998
4,Govind Nanda,1,first_serve,Serve,Slice,61.779400,2,1,1,short,ad,far,3.506049,16.034611,deep,ad,near,-0.379456,0.337840,2.605358,out wide,In,False,23:59:50,52.980000


### Load in Points data

In [9]:
swing_data_points = pd.read_excel(your_file_name, sheet_name='Points')
swing_data_points.shape

(86, 15)

In [10]:
def create_point(server, player1score, player2score):
    if server == "host":
        return str(player1score) + "-" + str(player2score)
    else:
        return str(player2score) + "-" + str(player1score)
    
swing_data_points['pointScore'] = swing_data_points.apply(lambda x: create_point(x['Match Server'], x['Host Game Score'], x['Guest Game Score']), axis=1)

In [11]:
swing_data_points = swing_data_points.rename(columns={'Break Point' : 'isBreakPoint'})
swing_data_points['isBreakPoint'] = swing_data_points['isBreakPoint'].replace(False, '')
swing_data_points['isBreakPoint'] = swing_data_points['isBreakPoint'].replace(True, 1)

In [12]:
swing_data_points = swing_data_points[['Point', 'pointScore', 'isBreakPoint']]
swing_data_points.head()

,Point,pointScore,isBreakPoint
0,1,0-0,
1,2,15-0,
2,3,15-15,
3,4,15-30,
4,5,15-40,1


In [13]:
swing_data = pd.merge(swing_data, swing_data_points, on='Point')

### Load in Games data

In [14]:
swing_data_games = pd.read_excel(your_file_name, sheet_name='Games')
swing_data_games.shape

(16, 9)

In [15]:
swing_data_games.head()

,Game,Set,Server,Host Set Score,Guest Set Score,Game Winner,Start Time,Video Time,Duration
0,1,1,host,0,0,guest,23:59:09,11.350000,159.800003
1,2,1,guest,0,1,guest,00:01:49,171.149994,261.700012
2,3,1,host,0,2,host,00:06:10,432.850006,148.710007
3,4,1,guest,1,2,guest,00:08:39,581.559998,223.429993
4,5,1,host,1,3,guest,00:12:22,804.989990,236.559998


In [16]:
def create_game(player1game, player2game):
        return str(player1game) + "-" + str(player2game)

    
swing_data_games['gameScore'] = swing_data_games.apply(lambda x: create_game(x['Host Set Score'], x['Guest Set Score']),  axis=1)

In [17]:
swing_data_games

,Game,Set,Server,Host Set Score,Guest Set Score,Game Winner,Start Time,Video Time,Duration,gameScore
0,1,1,host,0,0,guest,23:59:09,11.350000,159.800003,0-0
1,2,1,guest,0,1,guest,00:01:49,171.149994,261.700012,0-1
2,3,1,host,0,2,host,00:06:10,432.850006,148.710007,0-2
3,4,1,guest,1,2,guest,00:08:39,581.559998,223.429993,1-2
4,5,1,host,1,3,guest,00:12:22,804.989990,236.559998,1-3
5,6,1,guest,1,4,host,00:16:19,1041.560059,284.700012,1-4
6,7,1,host,2,4,guest,00:21:04,1326.250000,179.000000,2-4
7,8,1,guest,2,5,guest,00:24:03,1505.250000,285.000000,2-5
8,9,2,host,0,0,guest,00:28:48,1790.239990,441.679993,0-0
9,10,2,guest,0,1,guest,00:36:09,2231.919922,243.910004,0-1


In [18]:
swing_data_games = swing_data_games[['Game', 'gameScore']]

In [19]:
swing_data = pd.merge(swing_data, swing_data_games, on="Game")

In [20]:
swing_data.head()

,Player,Shot,Type,Stroke,Spin,Speed (MPH),Point,Game,Set,Bounce Depth,Bounce Zone,Bounce Side,Bounce (x),Bounce (y),Hit Depth,Hit Zone,Hit Side,Hit (x),Hit (y),Hit (z),Direction,Result,Favorited,Start Time,Video Time,pointScore,isBreakPoint,gameScore
0,Govind Nanda,1,first_serve,Serve,Kick,71.005798,1,1,1,deep,deuce_out,far,-6.920850,18.294973,deep,deuce,near,0.832136,0.170458,2.628215,out wide,Out,False,23:59:07,9.350000,0-0,,0-0
1,Cooper Willams,2,first_return,Forehand,Slice,27.993538,1,1,1,short,deuce,near,0.240642,9.327007,deep,deuce_alley,far,-4.775480,24.602375,1.123737,down the line,In,False,23:59:08,10.470000,0-0,,0-0
2,Govind Nanda,3,serve_plus_one,Forehand,Flat,36.418350,1,1,1,deep,deuce,far,-1.865687,18.531191,short,deuce,near,1.031473,5.433082,1.670364,cross court,In,False,23:59:11,13.150000,0-0,,0-0
3,Govind Nanda,1,first_serve,Serve,Flat,53.824200,2,1,1,short,deuce,near,2.591806,10.773574,deep,ad,near,-0.580245,0.338901,2.584491,out wide,Net,False,23:59:33,35.349998,15-0,,0-0
4,Govind Nanda,1,first_serve,Serve,Slice,61.779400,2,1,1,short,ad,far,3.506049,16.034611,deep,ad,near,-0.379456,0.337840,2.605358,out wide,In,False,23:59:50,52.980000,15-0,,0-0


### Load in Sets data

In [21]:
swing_data_sets = pd.read_excel(your_file_name, sheet_name='Sets')
swing_data_sets.shape

(2, 10)

In [22]:
swing_data_sets

,Set,Host Score,Guest Score,Host Tiebreak Score,Guest Tiebreak Score,Set Winner,Super Tiebreak,Start Time,Video Time,Duration
0,1,2,6,0,0,guest,False,23:59:09,11.35000,1778.890015
1,2,2,6,0,0,guest,False,00:28:48,1790.23999,2260.159912


In [23]:
host_set_score = 0
guest_set_score = 0

def create_set(set_winner):
        global host_set_score, guest_set_score  # Declare global variables
        if set_winner == "host":
                host_set_score += 1
        else:        
                guest_set_score += 1
        
        return str(host_set_score) + "-" + str(guest_set_score)

swing_data_sets['setScore'] = None
swing_data_sets.at[0, 'setScore'] = "0-0"
    
swing_data_sets.iloc[1:, swing_data_sets.columns.get_loc('setScore')] = swing_data_sets.iloc[1:].apply(lambda x: create_set(x['Set Winner']),  axis=1)


In [24]:
swing_data_sets = swing_data_sets[['Set', 'setScore']]


In [25]:
swing_data = pd.merge(swing_data, swing_data_sets, on="Set")
swing_data.head()

,Player,Shot,Type,Stroke,Spin,Speed (MPH),Point,Game,Set,Bounce Depth,Bounce Zone,Bounce Side,Bounce (x),Bounce (y),Hit Depth,Hit Zone,Hit Side,Hit (x),Hit (y),Hit (z),Direction,Result,Favorited,Start Time,Video Time,pointScore,isBreakPoint,gameScore,setScore
0,Govind Nanda,1,first_serve,Serve,Kick,71.005798,1,1,1,deep,deuce_out,far,-6.920850,18.294973,deep,deuce,near,0.832136,0.170458,2.628215,out wide,Out,False,23:59:07,9.350000,0-0,,0-0,0-0
1,Cooper Willams,2,first_return,Forehand,Slice,27.993538,1,1,1,short,deuce,near,0.240642,9.327007,deep,deuce_alley,far,-4.775480,24.602375,1.123737,down the line,In,False,23:59:08,10.470000,0-0,,0-0,0-0
2,Govind Nanda,3,serve_plus_one,Forehand,Flat,36.418350,1,1,1,deep,deuce,far,-1.865687,18.531191,short,deuce,near,1.031473,5.433082,1.670364,cross court,In,False,23:59:11,13.150000,0-0,,0-0,0-0
3,Govind Nanda,1,first_serve,Serve,Flat,53.824200,2,1,1,short,deuce,near,2.591806,10.773574,deep,ad,near,-0.580245,0.338901,2.584491,out wide,Net,False,23:59:33,35.349998,15-0,,0-0,0-0
4,Govind Nanda,1,first_serve,Serve,Slice,61.779400,2,1,1,short,ad,far,3.506049,16.034611,deep,ad,near,-0.379456,0.337840,2.605358,out wide,In,False,23:59:50,52.980000,15-0,,0-0,0-0


### Create shot data csv

In [26]:
# Check existing columns
swing_data.columns

Index(['Player', 'Shot', 'Type', 'Stroke', 'Spin', 'Speed (MPH)', 'Point',
       'Game', 'Set', 'Bounce Depth', 'Bounce Zone', 'Bounce Side',
       'Bounce (x)', 'Bounce (y)', 'Hit Depth', 'Hit Zone', 'Hit Side',
       'Hit (x)', 'Hit (y)', 'Hit (z)', 'Direction', 'Result', 'Favorited',
       'Start Time', 'Video Time', 'pointScore', 'isBreakPoint', 'gameScore',
       'setScore'],
      dtype='object')

In [27]:
swing_data.head()

,Player,Shot,Type,Stroke,Spin,Speed (MPH),Point,Game,Set,Bounce Depth,Bounce Zone,Bounce Side,Bounce (x),Bounce (y),Hit Depth,Hit Zone,Hit Side,Hit (x),Hit (y),Hit (z),Direction,Result,Favorited,Start Time,Video Time,pointScore,isBreakPoint,gameScore,setScore
0,Govind Nanda,1,first_serve,Serve,Kick,71.005798,1,1,1,deep,deuce_out,far,-6.920850,18.294973,deep,deuce,near,0.832136,0.170458,2.628215,out wide,Out,False,23:59:07,9.350000,0-0,,0-0,0-0
1,Cooper Willams,2,first_return,Forehand,Slice,27.993538,1,1,1,short,deuce,near,0.240642,9.327007,deep,deuce_alley,far,-4.775480,24.602375,1.123737,down the line,In,False,23:59:08,10.470000,0-0,,0-0,0-0
2,Govind Nanda,3,serve_plus_one,Forehand,Flat,36.418350,1,1,1,deep,deuce,far,-1.865687,18.531191,short,deuce,near,1.031473,5.433082,1.670364,cross court,In,False,23:59:11,13.150000,0-0,,0-0,0-0
3,Govind Nanda,1,first_serve,Serve,Flat,53.824200,2,1,1,short,deuce,near,2.591806,10.773574,deep,ad,near,-0.580245,0.338901,2.584491,out wide,Net,False,23:59:33,35.349998,15-0,,0-0,0-0
4,Govind Nanda,1,first_serve,Serve,Slice,61.779400,2,1,1,short,ad,far,3.506049,16.034611,deep,ad,near,-0.379456,0.337840,2.605358,out wide,In,False,23:59:50,52.980000,15-0,,0-0,0-0


In [28]:
# add in all desired column labels, with swingvision labels at end

columm_names = (['pointScore', 'gameScore', 'setScore',
                'isPointStart', 'pointStartTime', 'isPointEnd', 'pointEndTime','pointNumber',
                'isBreakPoint','shotInRally','side','serverName',
                'serverFarNear','firstServeIn','firstServeZone',
                'firstServeXCoord','firstServeYCoord',
                'secondServeIn','secondServeZone','secondServeXCoord',
                'secondServeYCoord','isAce','shotContactX',
                'shotContactY','shotDirection','shotFhBh',
                'isSlice','isVolley','isOverhead','isApproach','isDropshot', 'isLet',
                'isExcitingPoint','atNetPlayer1','atNetPlayer2','isLob',
                'shotLocationX','shotLocationY','isWinner','isErrorWideR', 'isErrorWideL',
                'isErrorNet','isErrorLong','clientTeam',
                'Date', 'Division', 'Event', 'lineupPosition','matchDetails',
                'matchVenue' , 'opponentTeam', 
                'player1Name', 'player2Name','player1Hand','player2Hand',
            'Round','Surface','Notes'])

shot_data = pd.DataFrame(columns=columm_names)
shot_data

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes


### Score Columns

In [29]:
shot_data['pointScore'] = swing_data['pointScore']
shot_data['gameScore'] = swing_data['gameScore']
shot_data['setScore'] = swing_data['setScore']

In [30]:
shot_data.head(10)

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes
0,0-0,0-0,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0-0,0-0,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0-0,0-0,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15-0,0-0,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15-0,0-0,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,15-0,0-0,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,15-15,0-0,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,15-15,0-0,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,15-15,0-0,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,15-30,0-0,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### isPointStart and isPointEnd columns 

In [31]:
def assign_pointstart(x):
    if (x == 'first_serve') | (x == 'second_serve'):
        return 1
    
    return ''

shot_data['isPointStart'] = swing_data['Type'].apply(assign_pointstart)


index_list = []

for i in swing_data['Point'].unique().tolist():
    last_point_index = swing_data[swing_data['Point'] == i].index[-1]
    index_list.append(last_point_index)
    
shot_data.loc[index_list,'isPointEnd'] = 1
shot_data['isPointEnd'] = shot_data['isPointEnd'].fillna('')

### pointStartTime and pointEndTime Columns

In [32]:
def convert_time(time):
    return int(time * 1000)

# def convert_time(time):
shot_data['pointStartTime'] = swing_data['Video Time'].apply(convert_time)

# Assigns last shot time to pointEndTime column
shot_data['pointEndTime'] = np.where(shot_data['isPointEnd'] == 1, shot_data['pointStartTime'], '')

### pointNumber Column

In [33]:
shot_data['pointNumber'] = swing_data['Point']

### isBreakPoint Column

In [34]:
shot_data['isBreakPoint'] = swing_data['isBreakPoint']

### shotInRally column

In [35]:
shot_data.shotInRally = swing_data.Shot

### side Column

In [36]:
def side(x, side, xcoord):
    if 'deuce' in x:
        return 'Deuce'
    elif 'ad' in x:
        return 'Ad'
    elif 'center_line' in x: # unique values include deuce, ad and center_line
        if (side == 'near') & (xcoord > 0):
            return 'Deuce'
        else:
            return 'Ad'
    else:
        return ''

shot_data['side'] = swing_data.apply(lambda x: side(x['Hit Zone'], x['Hit Side'], x['Bounce (x)']), axis = 1)

### Players

In [37]:
# ucla roster 23-24 men and womens
ucla_roster_23 = ["Gianluca Ballotta", 
                   "Jeffrey Fradkin", 
                   "Alexander Hoogmartens",
                   "Spencer Johnson",
                   "Stefan Leustian",
                   "Timothy Li",
                   "Govind Nanda",
                   "Jorge Plans Gonzalez",
                   "Giacomo Revelli",
                   "Aadarsh Tripathi",
                   "Emon van Loben Sels",
                   "Azuma Visaya",
                   "Rudy Quan",
                   "Leo Von Bismark",
                   
                   "Tian Fangran",
                   "Bianca Fernandez",
                   "Ahmani Guichard",
                   "Kimmi Hance",
                   "Mia Jovic",
                   "Anne-Christine Lutkemeyer",
                   "Vanessa Ong",
                   "Sasha Vagramov",
                   "Elise Wagle"]

In [38]:
# list of names who are playing in match
players = swing_data['Player'].unique()

# checks which one is UCLA player
is_ucla_player = [any([name in roster_name for roster_name in ucla_roster_23]) for name in players]

In [39]:
# assigns ucla player to player 1, and non ucla to player 2
shot_data.loc[0, "player1Name"] = players[is_ucla_player]
shot_data.loc[0, "player2Name"] = players[np.invert(is_ucla_player)]

### serverName Column

In [40]:
def assign_server_name(stroke, server):
    if stroke != 'Serve':
        return ''
    
    if server.startswith(players[is_ucla_player][0]):
        return 'Player1'
    elif server.startswith(players[np.invert(is_ucla_player)][0]):
        return 'Player2'
    
shot_data['serverName'] = swing_data.apply(lambda x: assign_server_name(x['Stroke'], x['Player']), axis=1)
shot_data['serverName'].replace(['', 'na'], pd.NaT, inplace=True)
shot_data['serverName'] = shot_data['serverName'].ffill()

### serverFarNear Column

In [41]:
shot_data.serverFarNear = np.where((swing_data.Stroke == 'Serve'), np.where(swing_data['Hit Side'] == 'far', 'Far', 'Near'), '')
shot_data['serverFarNear'].replace(['', 'na'], pd.NaT, inplace=True)
shot_data['serverFarNear'] = shot_data['serverFarNear'].ffill()

### firstServeIn and secondServeIn Columns

In [42]:
shot_data.firstServeIn = np.where((swing_data.Type == 'first_serve'),np.where((shot_data.isPointStart == 1) & (swing_data['Result'] == 'In'), 1, 0), np.nan)
shot_data.secondServeIn =np.where((swing_data.Type == 'second_serve') & (shot_data.isPointStart == 1), np.where(swing_data['Result'] == 'In', 1,0), np.nan)

### SwingVision Coord Transformation
court coordinates
swing vision - meters, near side center marks (0,0)
singles court x [-4.1148, 4.1148], y [0, 23.7744]
doubles court x [-5.485, 5.485]

our coordinates - center of net (0,0)
singles court x [-157.5, 157.5], y [-455, 455]

shot_x = (157.5/4.1148) * swing_x
shot_y = (455/11.8872) * swing_y + 455
ratio = 38.2764654418

### firstServeXCoord, firstYServeYCoord, secondServeXCoord, and secondServeyCoord Columns

In [43]:
def first_serve_x_coordinates(stroke, x):
    if stroke == 'first_serve':
        return x * 38.2764654418
    else:
        return np.nan

def first_serve_y_coordinates(stroke, y):
    if stroke == 'first_serve':
        return (y - 11.8872) * 38.2764654418
    else:
        return np.nan
    
shot_data['firstServeXCoord'] = swing_data.apply(lambda row: first_serve_x_coordinates(row['Type'], row['Bounce (x)']), axis=1)
shot_data['firstServeYCoord'] = swing_data.apply(lambda row: first_serve_y_coordinates(row['Type'], row['Bounce (y)']), axis=1)


def second_serve_x_coordinates(stroke, x):
    if stroke == 'second_serve':
        return x * 38.2764654418
    else:
        return np.nan

def second_serve_y_coordinates(stroke, y):
    if stroke == 'second_serve':
        return (y - 11.8872) * 38.2764654418
    else:
        return np.nan
    
shot_data['secondServeXCoord'] = swing_data.apply(lambda row: second_serve_x_coordinates(row['Type'], row['Bounce (x)']), axis=1)
shot_data['secondServeYCoord'] = swing_data.apply(lambda row: second_serve_y_coordinates(row['Type'], row['Bounce (y)']), axis=1)

### firstServeZone and secondServeZone Columns
- serving zones: T, Body, Wide
- Wide: x in [-inf, -105] u [105, inf]
- Body: x in [-105, -52.5] u [52.5, 105]
- T: x in [-52.5, 52.5]

In [44]:
def label_zone(x_coord):
    if x_coord != '':
        x_coord = float(x_coord)
        if (x_coord < -105) | (x_coord > 105):
            return 'Wide'
        elif (-105 <= x_coord <= -52.5) | (52.5 <= x_coord <= 105):
            return 'Body'
        elif -52.5 < x_coord < 52.5:
            return 'T'
    return ''

# convert x coord to serve zone
shot_data.firstServeZone = shot_data.firstServeXCoord.apply(label_zone)
shot_data.secondServeZone = shot_data.secondServeXCoord.apply(label_zone)

In [45]:
shot_data.head()

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes
0,0-0,0-0,0-0,1,9350,,,1,,1,Deuce,Player1,Near,0.0,Wide,-264.905676,245.266902,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Govind Nanda,Cooper Willams,NaN,NaN,NaN,NaN,NaN
1,0-0,0-0,0-0,,10470,,,1,,2,Deuce,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0-0,0-0,0-0,,13150,1,13150,1,,3,Deuce,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15-0,0-0,0-0,1,35349,,,2,,1,Ad,Player1,Near,0.0,Body,99.205173,-42.625667,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15-0,0-0,0-0,1,52980,,,2,,1,Ad,Player1,Near,1.0,Wide,134.199163,158.748234,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### isAce Column

In [46]:
shot_data['isAce'] = np.where((swing_data.Stroke == 'Serve') & 
                              (shot_data.isPointEnd == 1) & 
                              (shot_data.secondServeIn != 0), 1, np.nan)

### shotContactX and shotContactY Columns

In [47]:
# Functions to transform the swingvision coordinates
def transf_x_coord_sv_to_shot(sv_col) :
    return sv_col * 38.2764654418
def transf_y_coord_sv_to_shot(sv_col) :
    return (sv_col - 11.8872) * 38.2764654418

# want to convert swingvision coordinates into our own
shot_data['shotContactX'] = transf_x_coord_sv_to_shot(swing_data['Hit (x)'])
shot_data['shotContactY'] = transf_y_coord_sv_to_shot(swing_data['Hit (y)'])

### shotFhBh Column

In [48]:
def classify_shot(stroke):
    if stroke == 'FH Volley':
        return 'Forehand'
    elif stroke == 'BH Volley':
        return 'Backhand'
    elif stroke == 'Forehand':
        return 'Forehand'
    elif stroke == 'Backhand':
        return 'Backhand'
    elif stroke == 'Overhead':
        return 'Forehand'
    else:
        return ''

# Applying the function to the DataFrame
shot_data['shotFhBh'] = swing_data['Stroke'].apply(classify_shot)

### isSlice, isTopspin, isFlat, isKick Columns

In [49]:
shot_data['isSlice'] = swing_data['Spin'].apply(lambda x: '1' if x == 'Slice' else '')
shot_data['isTopspin'] = swing_data['Spin'].apply(lambda x: '1' if x == 'Topspin' else '') # added these metrics
shot_data['isFlat'] = swing_data['Spin'].apply(lambda x: '1' if x == 'Flat' else '') # added these metrics
shot_data['isKick'] = swing_data['Spin'].apply(lambda x: '1' if x == 'Kick' else '') # added these metrics

### isVolley Column

In [50]:
shot_data['isVolley'] = swing_data['Stroke'].apply(lambda x: 1 if x in ['FH Volley', 'BH Volley', 'Volley'] else '') # need to classify shotFhBh when doing isVolley

### isOverhead Column

In [51]:
shot_data['isOverhead'] = swing_data['Stroke'].apply(lambda x: 1 if x == 'Overhead' else '')

### isApproach Column

In [52]:
# maybe run model to predict 

# features to consider:
# player is inside the court

# Workflow:
# watch all points and tag all points that have _____
# subset df with points (testing x and y)


### isDropshot Column

In [53]:
# maybe run model to predict
# features to consider: 
# shotlocationY if close to the net
# shotContactY is close to the net
# speed of the ball (in swingvision data)

### isLet Column

In [ ]:
# maybe run model to predict OR get from swingvision data
# features to consider: 


### isExcitingPoint

In [54]:
# maybe run model to predict
# features to consider:
# rally length is long (maybe take _% percintile of rallies)
# point ends in a winner
# either player moves a lot
# amount of volleys, overheads
# breakpoint factor

### atNetPlayer1 and atNetPlayer2 Columns

In [55]:
# aggregated in STP

### isLob Column

In [56]:
# maybe run model to predict
# features to consider:
# opponent is at the net
# speed of the ball (in swingvision data)

### shotLocationX and shotLocationY Columns

In [57]:
# Functions to transform the swingvision coordinates
def transf_x_loc(stroke, sv_col):
    if stroke != 'first_serve' and stroke != 'second_serve':
        return sv_col * 38.2764654418
    return np.nan
    
def transf_y_loc(stroke, sv_col):
    if stroke != 'first_serve' and stroke != 'second_serve':
        return (sv_col - 11.8872) * 38.2764654418
    return np.nan

# want to convert swingvision coordinates into our own
shot_data['shotLocationX'] = swing_data.apply(lambda x: transf_x_loc(x['Type'], x['Bounce (x)']), axis=1)
shot_data['shotLocationY'] = swing_data.apply(lambda x: transf_y_loc(x['Type'], x['Bounce (y)']), axis=1)

### shotDirection column

In [58]:
# down the line --> switches btwn deuce and ad
# crosscourt --> remains on same side
shot_data['shotDirection'] = np.where((shot_data.shotContactX * shot_data.shotLocationX > 0) & (shot_data.shotInRally != 1), 
"Down the Line", 
    np.where((shot_data.shotInRally != 1), 'Crosscourt', ''))

### isWinner Column

In [59]:
shot_data.isWinner = np.where((shot_data.isPointEnd == 1) & (shot_data.secondServeIn != '0') &
                              (swing_data.Result == 'In'), 1, np.nan)


### isErrorWideR Column

In [60]:
def wide_right_function(side, x, y, end):
    if (side == 'far' and x < -157.5 and end == 1) or (side == 'near' and x > 157.5 and end == 1):
        return 1
    return np.nan

# Assign 'isErrorWideR' using values from both 'swing_data' and 'shot_data'
shot_data['isErrorWideR'] = shot_data.apply(lambda x: wide_right_function(swing_data.loc[x.name, 'Hit Side'], 
                                                                x['shotLocationX'], x['shotLocationY'], x['isPointEnd']), axis=1)


### isErrorWideL Column

In [61]:
def wide_left_function(side, x, y, end):
    if (side == 'far' and x > 157.5 and end == 1) or (side == 'near' and x < -157.5 and end == 1):
        return 1
    return np.nan

# Assign 'isErrorWideR' using values from both 'swing_data' and 'shot_data'
shot_data['isErrorWideL'] = shot_data.apply(lambda x: wide_left_function(swing_data.loc[x.name, 'Hit Side'], 
                                                                x['shotLocationX'], x['shotLocationY'], x['isPointEnd']), axis=1)


### isErrorNet Column

In [62]:
shot_data.isErrorNet = np.where((swing_data.Result == 'Net'), 1, np.nan)

### isErrorLong Column

In [63]:
shot_data['isErrorLong'] = np.where((swing_data['Result'] == 'Out') & (shot_data['shotLocationY'].abs() > 455), 1, np.nan)

### Group First Serve and Second Serve Columns

In [64]:
# All columns
default_cols = ['pointScore', 'gameScore', 'setScore', 'isPointStart', 'pointStartTime',
       'isPointEnd', 'pointEndTime', 'pointNumber', 'isBreakPoint',
       'shotInRally', 'side', 'serverName', 'serverFarNear', 'firstServeIn',
       'firstServeZone', 'firstServeXCoord', 'firstServeYCoord',
       'secondServeIn', 'secondServeZone', 'secondServeXCoord',
       'secondServeYCoord', 'isAce', 'shotContactX', 'shotContactY',
       'shotDirection', 'shotFhBh', 'isSlice', 'isVolley', 'isOverhead',
       'isApproach', 'isDropshot', 'isExcitingPoint', 'atNetPlayer1',
       'atNetPlayer2', 'isLob', 'shotLocationX', 'shotLocationY', 'isWinner',
       'isErrorWideR', 'isErrorWideL', 'isErrorNet', 'isErrorLong',
       'clientTeam', 'Date', 'Division', 'Event', 'lineupPosition',
       'matchDetails', 'matchVenue', 'opponentTeam', 'player1Name',
       'player2Name', 'player1Hand', 'player2Hand', 'Round', 'Surface',
       'Notes', 'isTopspin', 'isFlat', 'isKick']

# Assign all columns to have value be taken from the first serve row
agg_dict = {col: 'first' for col in default_cols}

# Reassign select columns to have value be taken form the second serve row
agg_dict.update({'isPointEnd': 'last', 
                 'pointEndTime': 'last', 
                 'secondServeIn' : 'last',
                 'secondServeZone' : 'last',
                 'secondServeXCoord' : 'last', 
                 'secondServeYCoord' : 'last',
                 'isAce' : 'last', 
                 'shotContactX' : 'last', 
                 'shotContactY' : 'last',
                 'isWinner' : 'last',
                 'isErrorWideR' : 'last',
                 'isErrorWideL' : 'last', 
                 'isErrorNet' : 'last',
                 'isErrorLong' : 'last'
                })

# Group by isPointStart and pointNumber
grouped_df = shot_data.groupby(['shotInRally', 'pointNumber'], as_index=False).agg(agg_dict)
shot_data = grouped_df.sort_values(by=['pointNumber', 'shotInRally'], ascending=[True, True]).reset_index(drop = True)

### Save as CSV

In [65]:
player1NameNoSpace = str(shot_data.iloc[0]['player1Name']).replace(" ", "")
player2NameNoSpace = str(shot_data.iloc[0]['player2Name']).replace(" ", "")

shot_data.to_csv(f'swingvision_{player1NameNoSpace}_{player2NameNoSpace}.csv', index=False)
print(f'swingvision_{player1NameNoSpace}_{player2NameNoSpace}.csv')

swingvision_GovindNanda_CooperWillams.csv


### Notebook End

## Errors in Swingvision Data Exploration

#### Chcek all the rows where isPointEnd != 1 and there is  isWinner, isErrorWideL, isErrorWideR, isErrorNet, isErrorLong

In [67]:
point_error = shot_data[(shot_data['isPointEnd'] != 1) & (shot_data['isPointStart'] != 1) &
          ((shot_data['isWinner'] == 1) | 
          (shot_data['isErrorNet'] == 1) | 
          (shot_data['isErrorLong'] == 1) |
          (shot_data['isErrorWideL'] == 1) |
          (shot_data['isErrorWideR'] == 1))]

point_error_numbers = point_error['pointNumber'].to_list()

if len(point_error) > 0:
    display(point_error)
    raise ValueError('Manually check points', point_error_numbers)

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes,isTopspin,isFlat,isKick
89,15-0,1-3,0-0,,856940,,,21,,3,Ad,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-46.499628,-384.373720,Crosscourt,Forehand,,,,None,None,None,None,None,None,124.938287,465.633087,NaN,NaN,NaN,NaN,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,,
133,0-0,2-4,0-0,,1363699,,,31,,2,Deuce,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,55.024142,-512.923469,Down the Line,Backhand,,,,None,None,None,None,None,None,1.943028,455.127652,NaN,NaN,NaN,NaN,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
219,15-0,0-1,0-1,,2356939,,,47,,7,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,24.936505,-155.016355,Down the Line,,,1,,None,None,None,None,None,None,157.292771,572.256584,NaN,NaN,NaN,NaN,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
230,0-0,0-2,0-1,,2500439,,,51,,3,Ad,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,58.993755,384.587533,Crosscourt,Backhand,,,,None,None,None,None,None,None,-71.522102,-496.681119,NaN,NaN,NaN,NaN,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,,
308,30-30,1-3,0-1,,3172830,,,67,,4,Ad,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,156.495664,226.716098,Down the Line,Backhand,,,,None,None,None,None,None,None,199.269537,222.168892,NaN,NaN,NaN,1.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,,
333,0-0,2-3,0-1,,3379020,,,70,,4,Ad,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,162.272578,506.627565,Down the Line,Backhand,1,,,None,None,None,None,None,None,113.281611,470.465147,NaN,NaN,NaN,1.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,,
369,15-15,2-4,0-1,,3724070,,,79,,6,Ad,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-45.151378,-482.689348,Crosscourt,Forehand,,,,None,None,None,None,None,None,129.492498,455.759520,NaN,NaN,NaN,NaN,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,,
383,30-30,2-4,0-1,,3890300,,,81,,4,Ad,Player2,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-106.091492,-465.347239,Crosscourt,Backhand,,,,None,None,None,None,None,None,40.827018,-41.247408,NaN,NaN,NaN,1.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,


ValueError: ('Manually check points', [21, 31, 47, 51, 67, 70, 79, 81])

In [76]:
shot_data[shot_data['pointNumber'] == 2]

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes,isTopspin,isFlat,isKick
3,15-0,0-0,0-0,1,35349,1,69970,2,,1,Ad,Player1,Near,0.0,Body,99.205173,-42.625667,0.0,T,3.78294,-182.774524,NaN,-13.142607,-442.920292,,,,,,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,1.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,


#### Check all the rows where there is isPointEnd == 1 but there is no isWinner, isErrorWideL, isErrorWideR, isErrorNet, isErrorLong
- Cj reccomendation: have this error check autmatically fill in how the point ends based on coordinate data

In [69]:
point_error = shot_data[(shot_data['isPointEnd'] == 1) &
                          (shot_data['isWinner'] != 1) &
                          (shot_data['isErrorWideL'] != 1) &
                          (shot_data['isErrorWideR'] != 1) &
                          (shot_data['isErrorNet'] != 1) & 
                          (shot_data['isErrorLong'] != 1) &
                          (shot_data['firstServeIn'] != 0) & 
                          (shot_data['secondServeIn'] != 0)]

point_error_numbers = point_error['pointNumber'].to_list()

if point_error.empty:
    print('Check Passed ✓')
else:

    display(point_error)
    raise ValueError('Manually check points', point_error_numbers)

Check Passed ✓


#### Volleys


In [91]:
shot_data.query('isVolley == 1')

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes,isTopspin,isFlat,isKick
13,15-40,0-0,0-0,,166270,,,5,1,5,Deuce,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,30.445254,-41.593313,Down the Line,,,1,,None,None,None,None,None,None,81.137150,321.410504,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
99,15-0,1-3,0-0,,870559,1,870559,21,,13,Deuce,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,109.099792,-120.824256,Down the Line,,,1,,None,None,None,None,None,None,255.087380,810.177045,NaN,1.0,NaN,NaN,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
105,15-15,1-3,0-0,,926859,1,926859,22,,6,Ad,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,80.570352,198.068843,Crosscourt,,,1,,None,None,None,None,None,None,-43.965382,3.457207,NaN,NaN,NaN,1.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
164,0-0,2-5,0-0,,1631349,,,35,,4,Ad,Player2,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-46.296036,-131.563369,Down the Line,,,1,,None,None,None,None,None,None,-136.689813,363.498502,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
166,0-0,2-5,0-0,,1634449,1,1634449,35,,6,Ad,Player2,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-48.706037,-69.452991,Down the Line,,,1,,None,None,None,None,None,None,-90.117995,209.273436,1.0,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
196,0-15,0-0,0-1,,2076939,1,2076939,41,,5,Ad,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-160.115737,-50.026919,Down the Line,,,1,,None,None,None,None,None,None,-48.237648,215.668515,1.0,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
212,0-0,0-1,0-1,,2317840,1,2317840,46,,4,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-59.008913,256.007355,Down the Line,,,1,,None,None,None,None,None,None,-94.731152,315.709263,NaN,NaN,NaN,1.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
217,15-0,0-1,0-1,,2355540,,,47,,5,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,29.210608,-131.969712,Crosscourt,,,1,,None,None,None,None,None,None,-89.226919,245.341656,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
219,15-0,0-1,0-1,,2356939,,,47,,7,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,24.936505,-155.016355,Down the Line,,,1,,None,None,None,None,None,None,157.292771,572.256584,NaN,NaN,NaN,NaN,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
307,30-30,1-3,0-1,,3171030,,,67,,3,Ad,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-76.386964,-137.598496,Crosscourt,,,1,,None,None,None,None,None,None,127.752258,225.651668,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,


In [93]:
shot_data[355:500]

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes,isTopspin,isFlat,isKick
355,0-0,2-4,0-1,1,3623590,,,77,,1,Deuce,Player1,Far,1.0,Wide,108.740491,-201.160089,NaN,,NaN,NaN,NaN,-32.517771,472.761581,,,,,,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
356,0-0,2-4,0-1,,3624419,1,3624419,77,,2,Deuce,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,163.385007,-508.552105,Down the Line,Forehand,,,,None,None,None,None,None,None,146.224404,241.133005,1.0,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,,
357,15-0,2-4,0-1,1,3650889,,,78,,1,Ad,Player1,Far,0.0,Body,-61.966918,1.771243,1.0,T,-46.523857,-187.289616,NaN,6.223677,431.001958,,,,,,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,1.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
358,15-0,2-4,0-1,,3670639,,,78,,2,Ad,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-49.961505,-357.821528,Crosscourt,Forehand,,,,None,None,None,None,None,None,152.617990,400.289419,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
359,15-0,2-4,0-1,,3672020,,,78,,3,Ad,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,163.004730,512.629353,Crosscourt,Backhand,,,,None,None,None,None,None,None,-25.769860,-265.882759,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,,
360,15-0,2-4,0-1,,3673169,,,78,,4,Ad,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-50.974606,-468.526176,Down the Line,Forehand,,,,None,None,None,None,None,None,-147.412735,240.044040,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,,
361,15-0,2-4,0-1,,3674750,,,78,,5,Deuce,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-197.972922,555.386538,Down the Line,Forehand,,,,None,None,None,None,None,None,-76.803068,-76.985455,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
362,15-0,2-4,0-1,,3676419,,,78,,6,Deuce,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,28.517613,-163.356797,Down the Line,,,1,,None,None,None,None,None,None,0.360909,398.554998,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
363,15-0,2-4,0-1,,3677389,1,3677389,78,,7,Ad,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,90.760351,465.265174,Crosscourt,Backhand,,,,None,None,None,None,None,None,-205.742011,-496.280556,NaN,1.0,NaN,NaN,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
364,15-15,2-4,0-1,1,3717419,,,79,,1,Deuce,Player1,Far,1.0,Wide,156.376241,-166.451219,NaN,,NaN,NaN,NaN,-33.577685,424.522135,,,,,,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,


#### Overheads

In [72]:
shot_data.query('isOverhead == 1')

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes,isTopspin,isFlat,isKick
221,15-0,0-1,0-1,,2360239,1,2360239,47,,9,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,118.944193,-252.597649,Crosscourt,Forehand,,,1,None,None,None,None,None,None,-160.746265,454.59982,NaN,NaN,1.0,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,


#### Aces
- WARNING: Not accurate
- FIX: counts double faults as aces

In [73]:
shot_data.query('isAce == 1')

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes,isTopspin,isFlat,isKick
79,30-0,1-2,0-0,1,775929,1,775929,18,,1,Deuce,Player2,Far,1.0,T,1.201996,-227.85134,NaN,,NaN,NaN,1.0,-48.704735,429.271479,,,,,,None,None,None,None,None,None,NaN,NaN,1.0,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,


#### Double Faults

In [88]:
shot_data.query('firstServeIn == 0').query('secondServeIn == 0')

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes,isTopspin,isFlat,isKick
3,15-0,0-0,0-0,1,35349,1,69970,2,,1,Ad,Player1,Near,0.0,Body,99.205173,-42.625667,0.0,T,3.782940,-182.774524,NaN,-13.142607,-442.920292,,,,,,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,1.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
34,40-15,0-1,0-0,1,370380,,,10,,1,Deuce,Player2,Near,0.0,T,11.307059,-25.638840,0.0,Body,-52.542295,263.234181,NaN,49.729741,-446.557169,,,,,,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,1.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
106,30-15,1-3,0-0,1,955270,1,964309,23,,1,Ad,Player1,Near,0.0,Body,97.435154,-42.945888,0.0,T,33.428560,-44.775312,NaN,-28.205430,-471.140573,,,1,,,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,1.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,,
265,40-40,0-2,0-1,1,2710100,1,2719899,57,1,1,Ad,Player1,Far,0.0,Wide,-191.688807,-220.564955,0.0,T,47.150252,-248.594198,NaN,32.941262,456.425760,,,,,,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,


In [85]:
shot_data[shot_data['pointNumber'] == 10]

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes,isTopspin,isFlat,isKick
34,40-15,0-1,0-0,1,370380,,,10,,1,Deuce,Player2,Near,0.0,T,11.307059,-25.63884,0.0,Body,-52.542295,263.234181,NaN,49.729741,-446.557169,,,,,,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,1.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
35,40-15,0-1,0-0,,383809,,,10,,2,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-73.238955,498.070669,Crosscourt,Backhand,,,,None,None,None,None,None,None,25.331097,-360.356655,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
36,40-15,0-1,0-0,,384850,,,10,,3,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,15.186647,-469.531966,Crosscourt,Forehand,,,,None,None,None,None,None,None,-78.026230,406.023502,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,,
37,40-15,0-1,0-0,,386179,,,10,,4,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-73.286341,512.967984,Crosscourt,Forehand,,,,None,None,None,None,None,None,87.543980,-327.437019,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,,
38,40-15,0-1,0-0,,387649,,,10,,5,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,115.323239,-518.601132,Crosscourt,Forehand,,,,None,None,None,None,None,None,-142.419341,345.483607,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,,
39,40-15,0-1,0-0,,388850,,,10,,6,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-176.819434,536.631299,Crosscourt,Forehand,,,,None,None,None,None,None,None,112.653456,-386.184542,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
40,40-15,0-1,0-0,,390079,,,10,,7,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,99.680184,-492.940322,Down the Line,Forehand,,,,None,None,None,None,None,None,45.869904,311.200372,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,,
41,40-15,0-1,0-0,,391660,1,391660,10,,8,Ad,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,44.211806,479.083858,Crosscourt,Backhand,,,,None,None,None,None,None,None,-96.317635,-75.251187,1.0,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,,


### Check all points where double fault occurs (firstServeIn == 0 & secondServeIn == 0) but len(shotInRally) > 1
- Check double fault but the point continues

### Check all the points where everytime the server changes, the first pointScore should be "0-0". If not output error
- Govind Nanda vs Cooper Williams (Harvard) row 380

### Points

In [ ]:
# # ad scoring?

# ad_scoring = False

In [ ]:
# # want to record the score every time a point ends
# # points: server - returner
# # games: ucla (player1) - opp
# # sets: ucla (player1) - opp
# points = np.zeros(2)
# games = np.zeros(2)
# sets = np.zeros(2)
# pt_values = [0, 15, 30, 40]



# shot_data.loc[0,"pointScore"] = f"{pt_values[int(points[0])]} - {pt_values[int(points[1])]}"
# shot_data.loc[0,"gameScore"] = f"{games[0]} - {games[1]}"
# shot_data.loc[0,"setScore"] = f"{sets[0]} - {sets[1]}"

# shot_data["isBreakPoint"] = ''

# error_cols = [x for x in shot_data.columns if "isError" in x]

# for i in range(0, len(shot_data.pointScore) - 1):
#     if shot_data.loc[i+1, "isPointStart"] == 1: # means we gotta update pts
#         # determine point score by checking last shot
#         if shot_data.loc[i, "isWinner"] == "1":
#             # check if player 1 or 2 won pt
#             pt_winner_player_num = (np.where(shot_data.loc[i, "Player"] == shot_data.loc[0,"player1Name"], '1', '2'))
#         elif shot_data.loc[i, 'secondServeIn'] == "0": # double fault
#             pt_winner_player_num = (np.where(shot_data.loc[i, "Player"] == shot_data.loc[0,"player1Name"], '2', '1'))
#         elif any(shot_data.loc[i,error_cols] == "1"):
#             # winner is the player who did NOT hit that shot
#             pt_winner_player_num = (np.where(shot_data.loc[i, "Player"] == shot_data.loc[0,"player1Name"], '2', '1'))
#         else:
#             print("no pt recorded at row ", i)

#         if shot_data.loc[i, "serverName"] is not None:
#             didServerWinPt = shot_data.loc[i, "serverName"][-1] == pt_winner_player_num
#         else:
#             print(f"Server name is None at row {i}. Skipping this point.")
#             continue  # Skip this point if server name is None
        
#         if didServerWinPt:
#             points[0] += 1
#         else:
#             points[1] += 1


#         if ad_scoring: # checks if need to win by 2
#             if any(points > 3) and abs(points[0] - points[1]) >= 2:
#                 game_winner = np.argmax(points)  # Find who won the game
#                 games[game_winner] += 1
#                 points = np.zeros(2)  # Reset point values
#             if any(games > 5) and abs(games[0] - games[1]) >= 2:
#                 set_winner = np.argmax(games)  # Find who won the set
#                 sets[set_winner] += 1
#                 games = np.zeros(2)  # Reset game values   
#         else:
#             if points[1] == 3: # if the returner has 40 pts and can win the game
#                 shot_data.loc[i + 1, 'isBreakPoint'] = '1'
#             if any(points > 3):
#                 game_winner = np.argmax(points)  # Find who won the game
#                 games[game_winner] += 1
#                 points = np.zeros(2)  # Reset point values
#             if any(games > 5):
#                 set_winner = np.argmax(games)  # Find who won the set
#                 sets[set_winner] += 1
#                 games = np.zeros(2)  # Reset game values   

#     # Update the scores in the shot_data DataFrame
#     shot_data.loc[i+1,"pointScore"] = f"{pt_values[int(points[0])]} - {pt_values[int(points[1])]}"
#     shot_data.loc[i+1,"gameScore"] = f"{int(games[0])} - {int(games[1])}"
#     shot_data.loc[i+1,"setScore"] = f"{int(sets[0])} - {int(sets[1])}"


# # Additional comments for further updates:
# # - Tiebreak scenarios are not yet handled and need to be accounted for in future versions.